# Shortest path

Shortest path between two stations via any starting/finishing platform

In [ ]:
from zone1tube.data.stations import ExtStations
import numpy as np


with open("../zone1.json", "r") as f:
    stations = ExtStations.validate_json(f.read())


station_lookup = {s.id: s.name for s in stations}
line_lookup = {d.line_ref: d.line_name for s in stations for d in s.destinations}

In [ ]:
from zone1tube.algorithms.graph import generate_graph


graph = generate_graph(stations, 7)

Calculate all shortest paths

In [ ]:
from scipy.sparse.csgraph import shortest_path


dists, predecessors = shortest_path(
    graph.csr_m, directed=True, return_predecessors=True
)

From the starting and finish station map to ids

In [ ]:
from_station = "Liverpool Street"
to_station = "Bayswater"
from_id = next(s.id for s in stations if s.name == from_station)
to_id = next(s.id for s in stations if s.name == to_station)

from_platforms = [graph.node_inverse_lookup[n] for n in graph.platform_lookup[from_id]]
to_platforms = [graph.node_inverse_lookup[n] for n in graph.platform_lookup[to_id]]

Find the platforms at the start and the shortest times to get to a platform at the destination

In [ ]:
shortest_from_tos = [
    x
    for x in (
        (graph.node_lookup[i],)
        + min(
            *((graph.node_lookup[j], dists[i, j]) for j in to_platforms),
            key=lambda x: x[1],
        )
        for i in from_platforms
    )
    if x[2] < np.inf
]
shortest_from_tos

Reconstruct the routes for each path

In [ ]:
def reconstruct_path(i: int, j: int, path=None):
    if path is None:
        path = []
    previous = predecessors[i, j]
    if previous == i:
        return (path + [j, i])[::-1]
    return reconstruct_path(i, previous, path=path + [j])


def node_to_str(n):
    station_id, line_ref = n
    return f"{station_lookup[station_id]} -> {line_lookup[line_ref]}"


for route, _ in sorted(
    [
        (
            "\n".join(
                node_to_str(graph.node_lookup[s])
                for s in reconstruct_path(
                    graph.node_inverse_lookup[from_n], graph.node_inverse_lookup[to_n]
                )
            )
            + f": {dist} min\n",
            dist,
        )
        for from_n, to_n, dist in shortest_from_tos
    ],
    key=lambda x: x[1],
):
    print(route)